In [1]:
#Need to add parent directoy to sys.path to find 'metadataDB'
import sys
sys.path.append('../')

%matplotlib inline
# import matplotlib.pyplot as plt 
import time
# import numpy as np
# import scipy as sp
import re
from collections import Counter
import random

# Natural language processing toolkit
# To use this, run nltk.download() and download 'stopwords'
# from nltk.corpus import stopwords
# s=stopwords.words('english') + ['']

# Machine learning
# from sklearn.feature_extraction.text import CountVectorizer
# from sklearn.feature_extraction.text import TfidfTransformer
# from sklearn.naive_bayes import MultinomialNB
# from sklearn.linear_model import LogisticRegression
# from sklearn.svm import LinearSVC, SVC
# from sklearn.pipeline import Pipeline
# from sklearn import metrics

# SQL
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker
from metadataDB.declareDatabase import *
from sqlalchemy import or_, and_

engine = create_engine("sqlite:///../arXiv_metadata.db", echo=False)
Base.metadata.bind = engine
DBsession = sessionmaker(bind=engine)
session = DBsession()

In [2]:
query = session.query(Article_Category)\
                    .join(Category)\
                    .join(Article)\
                    .filter(Category.name.like('%atom-ph'))
resultPRL = query.all()
abstract_all = [x.article.abstract for x in query.all()]
session.close_all()

In [3]:
abstract_train = abstract_all
print len(abstract_train)

9156


CountVectorizer might do a cleaner job.

In [4]:
class AbstractWriter:
    '''This class will take a specific n-gram and learn to write abstracts from a few words.'''
    _data = dict()
    _abstracts = []
    
    def __init__(self, ngram, randomize=False, seed=None, maxWords=200):
        self.ngram = ngram
        self.randomize = randomize
        self.seed = seed
        self.maxWords = maxWords
        if seed is not None:
            random.seed(seed)

    
    def fit(self, train):
        data_list = []
        for item in train:
            ind = 0
            textlist = self.__cleanup_text(item)
            self._abstracts.append(' '.join(textlist))
            while ind + self.ngram - 1 < len(textlist):
                currentTuple = tuple(textlist[ind:(ind+self.ngram-1)])
                nextWord = textlist[ind+self.ngram-1]
                data_list.append((tuple(textlist[ind:(ind+self.ngram-1)]),
                                        textlist[ind+self.ngram-1]))
                ind += 1
        data_list.sort()
        
        # Now that the list is sorted, convert it into a dictionary.
        # Since the list is sorted, each subsuquent entry is either new or a continuation of the previous one.

        previousKey = None
        for val in data_list:
            currentKey = val[0]
            if currentKey == previousKey:
                self._data[currentKey].append(val[1])
            else:
                self._data[currentKey] = [val[1]]
            previousKey = val[0]

    def find_next_word(self, phrase):
        '''phrase is a tuple of (typ. 3) words. Returns the next word.'''
        # Since data is sorted, it should be possible to search quickly. I'm not doing this properly.
        if phrase in self._data.keys():
            solution = self._data[phrase]
            
            # If there are more than one result, we either return the most popular option or
            # a random answer weighted by popularity.
            if self.randomize:
                return random.choice(solution)
            else:
                count = Counter(solution)
                return count.most_common(1)[0][0]
        else:
            return None
        
    def write_abstract(self, beginning):
        abstract = list(beginning)
        nextWord = ''
        ind = len(beginning) - self.ngram + 1
        while (nextWord is not None) and (ind < self.maxWords):
            nextWord = self.find_next_word(tuple(abstract[ind:(ind+self.ngram)]))
            if nextWord is None:
                break
            else:
                abstract.append(nextWord)
                ind += 1
        return ' '.join(abstract)
    
    def find_similar(self, abstract):
        '''Have I accidentally plagarized one of the articles?
        With too few choices, we will likely just copy an existing article.
        Returns the longest chunk highlighted in an existing article.'''
        first = 0
        last = 1
        abort = False
        longestChunk = ''
        longestChunkLength = 0

        totalLength = len(abstract.split())
        while first + longestChunkLength < totalLength:
            chunk = ' '.join(abstract.split()[first:last])
            if any(x for x in self._abstracts if chunk in x):
                if last - first + 1 > longestChunkLength:
                    longestChunk = chunk
                    longestChunkLength = last - first + 1
                last += 1
            else:
                first += 1
                last = first + 1
        
        best_abstract = (x for x in self._abstracts if longestChunk in x).next()
        return re.sub('(' + longestChunk + ')', '[[[' + longestChunk + ']]]', best_abstract)


    def __cleanup_text(self, text):
        # re from kaggle https://www.kaggle.com/c/word2vec-nlp-tutorial/details/part-1-for-beginners-bag-of-wordsi
        # Remove punctuation. Keep periods.
        # 
        # New trick: replace numbers with ##, which we will later replace with different numbers.
        pattern1 = re.compile("[0-9]")
        pattern2 = re.compile("[^a-zA-Z#.]")
        # The previous pattern could make multiple spaces. Turn multiple spaces into a single space.
        pattern3 = re.compile("(\s{2,})")
        return pattern3.sub(" ",
                            pattern2.sub(" ",
                                         pattern1.sub("#",
                                                      text.lstrip().rstrip()))).split(' ')

Let's make a bunch of abstracts that start in a way similar to existing abstracts, and find if they accidentally plagarize anything.

In [5]:
def WriteAbstract(ngram):
    writer = AbstractWriter(ngram=ngram, randomize=True, seed=42)
    writer.fit(abstract_train)
    for i in range(0,5):
        phrase = random.choice(writer._abstracts).split()[0:(ngram-1)]
        currentAbstract = writer.write_abstract(tuple(phrase))
        print 'New abstract: ' + currentAbstract
        print ''
        print 'Existing abstract: ' + writer.find_similar(currentAbstract)
        print ''
    return writer

In [6]:
WriteAbstract(4)

New abstract: An all optical atomic clock applications. We find that an impurity with bare charge Z # which includes weakly bound negative ions of Tl are predicted.

Existing abstract: We calculate exactly the vacuum polarization charge density in the field of a subcritical Coulomb impurity Z e r in graphene. Our analysis is based on the exact electron Green s function obtained by using the operator method and leads to results that are exact in the parameter Z alpha where alpha is the fine structure constant of graphene. Taking into account also electron electron interactions in the Hartree approximation we solve the problem self consistently in the subcritical regime where the impurity has an effective charge Z eff determined by the localized induced charge. [[[We find that an impurity with bare charge Z #]]] remains subcritical Z eff alpha # # for any alpha while impurities with Z # # and higher can become supercritical at certain values of alpha .

New abstract: We present a method 

<__main__.AbstractWriter instance at 0x103c70710>

In [7]:
WriteAbstract(5)

New abstract: Polar molecules in selected quantum states can be trapped in different trap types thereby enabling the realization of ultra precise ion clocks and quantum information processing applications. Extending this sensitivity to broad transitions is challenging due to the difficulty of detecting the rapid photon scattering events in this case. Here we demonstrate a large conditional cross phase modulation between a signal field stored inside an atomic quantum memory for non classical states of light operating at a wavelength compatible with standard telecom fibre infrastructure nor a fibre based implementation of a quantum memory for photons with fluorescence based quantum state measurement it is in principle possible in a static system. It is related to the number of qubits with controller Charlie.

Existing abstract: The ability to detect the interaction of light and matter at the single particle level is becoming increasingly important for many areas of science and technology

<__main__.AbstractWriter instance at 0x109f08560>

In [8]:
WriteAbstract(6)

New abstract: The recently developed notion of parity time PT symmetry in optical systems with a controlled gain loss interplay has spawned an intriguing way of achieving optical behaviors that are presently unattainable with standard arrangements. In most experimental studies so far however the implementations rely highly on the advances of nanotechnologies and sophisticated fabrication techniques to synthesize solid state materials. Here we report the first spectroscopic observation of the resonant dipole dipole interaction between two Rydberg atoms depends on the relative orientation of the atoms and on the change in the magnetic quantum number. We simulate the effect of this anisotropy on the energy transport in an amorphous many atom system subject to a homogeneous applied electric field. We consider two experimentally feasible geometries and find that the effects should be measurable in current generation imaging experiments. In both geometries atoms of p character are localized 

<__main__.AbstractWriter instance at 0x109b8d290>